### 1. Load libraries

In [6]:
import synapseclient
from scoremtb.score import compute_score as cs
from scoremtb.score import get_syndata as gsyn
import pandas as pd

### 2. Load dataset

In [2]:
PARQUET_FOLDER_ID = 'syn29357579'

In [7]:
# Load data from S3
syn = synapseclient.login()
dfid = gsyn.parquet_2_df(syn, PARQUET_FOLDER_ID, 'dataset_archivemetadata_v1/')
stepdata = gsyn.parquet_2_df(syn, PARQUET_FOLDER_ID, 'dataset_sharedschema_v1_steps/')
task_data = gsyn.parquet_2_df(syn, PARQUET_FOLDER_ID, 'dataset_sharedschema_v1/')
task_status = gsyn.parquet_2_df(syn, PARQUET_FOLDER_ID, 'dataset_sharedschema_v1_taskStatus/')


Welcome, Larsson Omberg!



INFO:synapseclient_default:Welcome, Larsson Omberg!



In [8]:
#Merge task status
task_status = task_status.drop(columns=['id', 'index'])
dfid = dfid.merge(task_status, on = ['recordid', 'assessmentid', 'year', 'month', 'day'],
                    how='left').reset_index(drop=True)

### 3. Filter Scores

##### 3.1 Study membership details

In [28]:
table_syn = syn.tableQuery('SELECT healthCode, participantVersion, studyMemberships FROM syn29300177')
study_reference = (table_syn.asDataFrame()
                     .sort_values(by=['healthCode', 'participantVersion'])
                     .reset_index(drop=True)
                     .loc[:,['healthCode','studyMemberships']]
                     .rename(columns={"healthCode": "healthcode"})
                     .drop_duplicates(keep='last').reset_index(drop=True)
                )


#### 3.2 Spelling

In [29]:
spelling_score = cs.get_score(task_data, stepdata, dfid, 'spelling', study_reference)

#### 3.3 Vocab

In [30]:
vocab_score = cs.get_score(task_data, stepdata, dfid, 'vocabulary', study_reference)

#### 3.4 PSM

In [31]:
PSM_score = cs.get_score(task_data, stepdata, dfid, 'psm', study_reference)

[WARNING] /Users/lom/Dropbox/Sage/mobile/mobile_toolbox/MTB_scoring/scoremtb/score/score_commons.py:197: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psm_filter[config[assmnt_val + '_score'][0]] = psm_filter.loc[:, config[assmnt_val + '_adjacentp']].sum(axis=1)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  psm_filter[config[assmnt_val + '_score'][0]] = psm_filter.loc[:, config[assmnt_val + '_adjacentp']].sum(axis=1)

[WARNING] /Users/lom/Dropbox/Sage/mobile/mobile_toolbox/MTB_scoring/scoremtb/score/score_commons.py:198: SettingWi

#### 3.5 MFS

In [32]:
score_mfs = cs.get_score(task_data, stepdata, dfid, 'memory-for-sequences', study_reference)

#### 3.6 DCCS

In [33]:
score_dccs = cs.get_score(task_data, stepdata, dfid, 'dccs', study_reference)

#### 3.7 Fname

In [34]:
score_fname = cs.get_score(task_data, stepdata, dfid, 'fnameb', study_reference)

#### 3.8 Number Match

In [35]:
score_nm = cs.get_score(task_data, stepdata, dfid, 'number-match', study_reference)

#### 3.9 Flanker

In [36]:
score_flanker = cs.get_score(task_data, stepdata, dfid, 'flanker', study_reference)

### 4. Combine Score

In [38]:
stack_merged = cs.combine_scores([score_mfs, score_dccs, score_fname, score_nm, score_flanker, PSM_score, 
                                   spelling_score, vocab_score], dfid)

In [39]:
stack_merged

,healthcode,recordid,assessmentid,deviceInfo,startDate,endDate,participantversion,clientinfo,sessionguid,taskStatus_val,sessionstarteventid,scoreId,score,study_reference,participant_id,dataGroups
0,cY4HIctb-HGzdJBNNpq5wF9k,SHmXD5jtFlEZoKnV6D_OIk8a,memory-for-sequences,iPhone XS; iOS/15.4.1,2022-05-30T09:13:00.733-05:00,2022-05-30T09:18:20.329-05:00,2,"{ ""appName"" : ""mobile-toolbox"", ""appVersio...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,mfs_sum,11.000000,fmqcjv,6sytfu,
1,lu00q1bRwYr51kwskzjR4gIg,damKL3weFYj3PeujGk9AMDza,memory-for-sequences,winglm LM-F100; Android/11,2022-05-30T00:53:00.809Z,2022-05-30T00:59:50.793Z,1,"{ ""appName"" : ""mobile-toolbox"", ""appVersio...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,mfs_sum,13.000000,fmqcjv,eyhtxh,
2,LKE_vT7FqlqsIyMpORfGJ8XI,eUpDXeaH3gxzCGlYp1XJv7Jp,memory-for-sequences,iPhone 8; iOS/15.5,2022-05-29T22:08:19.030-04:00,2022-05-29T22:11:38.886-04:00,2,"{ ""appName"" : ""mobile-toolbox"", ""appVersio...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,mfs_sum,2.000000,fmqcjv,ybkp6s,
3,FB5W5lycWo-t4CITY3EMwWkV,t08yjcYVFfOoibS7vO_MEE_W,memory-for-sequences,crownqltesq SM-N960U; Android/10,2022-05-30T00:25:32.488Z,2022-05-30T00:30:33.884Z,1,"{ ""appName"" : ""mobile-toolbox"", ""appVersio...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,mfs_sum,13.000000,fmqcjv,uk43h7,
4,l-sAnwL4YwRgEp2HCrz8GB4b,6Cr3gtA4yczVh-UQwrA4kRQf,memory-for-sequences,iPhone 11; iOS/16.0.2,2022-10-18T20:45:56.281-07:00,2022-10-18T20:52:37.214-07:00,2,"{ ""appName"" : ""mobile-toolbox"", ""appVersio...",fvAIJs5Aqaimp9eWCjwuINQj,completed,custom:start_m6_fwd,mfs_sum,10.000000,fmqcjv,tkyzx9,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74307,HWbvHWpiLOkOwKdvOapK_NGy,HItbsDjq2Co68NGUZJFQa8uE,vocabulary,"iPhone13,2; iOS/15.3.1",2022-03-02T23:28:14.099-07:00,2022-03-02T23:28:48.593-07:00,2,"{ ""appName"" : ""MobileToolboxApp"", ""appVers...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,vocab_se,0.553688,fmqcjv,yfhkzk,
74308,JEhyPP-ABCE7-BmoqdEKcJps,HLJmVLSqMCHzbHTu65Rt2tXo,vocabulary,"iPhone14,5; iOS/15.3.1",2022-03-15T09:53:32.720-04:00,2022-03-15T09:56:41.172-04:00,2,"{ ""appName"" : ""MobileToolboxApp"", ""appVers...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,vocab_theta,0.024429,fmqcjv,f6ep6p,
74309,JEhyPP-ABCE7-BmoqdEKcJps,HLJmVLSqMCHzbHTu65Rt2tXo,vocabulary,"iPhone14,5; iOS/15.3.1",2022-03-15T09:53:32.720-04:00,2022-03-15T09:56:41.172-04:00,2,"{ ""appName"" : ""MobileToolboxApp"", ""appVers...",K4pQ1FkyPodm_6XgUF4AlvJt,completed,enrollment,vocab_se,0.499984,fmqcjv,f6ep6p,
74310,l4FXJ3Gvi3-Z1JeGjhKEL_W5,HOv4NWO98UFkl9aEMPZ-gCq-,vocabulary,"iPhone14,2; iOS/15.4.1",2022-05-14T16:08:52.478-07:00,2022-05-14T16:12:24.779-07:00,2,"{ ""appName"" : ""MobileToolboxApp"", ""appVers...",rTKC_5nbBbNcM0WPAwS_Xk5j,completed,custom:start_d21_rev,vocab_theta,0.124767,fmqcjv,47epqu,
